In [1]:
%load_ext dotenv

In [2]:
%dotenv

In [ ]:
# necessary libraries
from langchain.memory import ConversationSummaryMemory
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from operator import itemgetter
from langchain_core.runnables import chain

# RAG setup libraries
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_text_splitters.markdown import MarkdownHeaderTextSplitter
from langchain_text_splitters.character import CharacterTextSplitter
from langchain_community.document_loaders import Docx2txtLoader, PyPDFLoader


page = PyPDFLoader('User Query.pdf')
my_document = page.load()

#header_splitter = MarkdownHeaderTextSplitter(
   # headers_to_split_on=[("#", "Course Title"), ("##", "Lecture Title")]
#)
#header_splitted_document = header_splitter.split_text(my_document[0].page_content)
#for i in range(len(header_splitted_document)):
    #header_splitted_document[i].page_content = ' '.join(header_splitted_document[i].page_content.split())

character_splitter = CharacterTextSplitter(separator=".", chunk_size=500, chunk_overlap=50)
character_splitted_documents = character_splitter.split_documents(my_document)
for i in range(len(character_splitted_documents)):
    character_splitted_documents[i].page_content = ' '.join(character_splitted_documents[i].page_content.split())

embedding = OpenAIEmbeddings(model="text-embedding-ada-002")
vector_store = Chroma.from_documents(
    embedding=embedding,
    documents=character_splitted_documents,
    persist_directory="./TCP_directory_1"
)

retriever = vector_store.as_retriever(search_type='mmr', search_kwargs={'k': 3, 'lambda_multi': 0.5})

#chatbot memory
chat_memory = ConversationSummaryMemory(llm=ChatOpenAI(), memory_key='message_log')

#prompt template for the chatbot
TEMPLATE = """
The AI should understand the question completely and only answer strictly based on the provided document context, but it should not talk about document in the answer.  
The AI should not hallucinate.
if the question does not have enough information, the AI should ask for more details.
The AI should Deliver well-structured, polished responses that enhance the overall user experience.
If the answer is not available in the context, the AI truthfully responds: "Sorry, I don't know the answer."

Current Conversation:
{message_log}

Question Context:
{context}

Human:
{question}

AI:
"""
prompt_template = PromptTemplate.from_template(template=TEMPLATE)

# ChatOpenAI instance
chat = ChatOpenAI(
    model="gpt-4",
    temperature=0,
    max_tokens=250
)

#chain combining memory and RAG
@chain
def memory_rag_chain(question):
    # Retrieve relevant documents from the vector store
    retrieved_docs = retriever.invoke(question)
    context = "\n".join([doc.page_content for doc in retrieved_docs])

    # If no relevant context is found, a "Sorry" response is returned
    if not context.strip():
        response = "Sorry, I don't know the answer."
        chat_memory.save_context(inputs={'input': question}, outputs={'output': response})
        return response

    # Combining memory and RAG for the prompt
    chain = (
        RunnablePassthrough.assign(
            message_log=RunnableLambda(chat_memory.load_memory_variables) | itemgetter("message_log"),
            context=RunnablePassthrough()  # Pass the RAG context
        )
        | prompt_template
        | chat
        | StrOutputParser()
    )

    # Invoking the chain
    response = chain.invoke({'question': question, 'context': context})

    # Saving the interaction in memory
    chat_memory.save_context(inputs={'input': question}, outputs={'output': response})

    return response

# Usage
if __name__ == "__main__":
    print("Chatbot is ready! Type 'exit' to end the conversation.")
    while True:
        user_input = input("You: ")
        if user_input.lower() in ['exit', 'quit']:
            break
        response = memory_rag_chain.invoke(user_input)
        print(f"Taha's Bot: {response}")



C:\Users\DELL\AppData\Local\Temp\ipykernel_9540\2571472939.py:44: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  chat_memory = ConversationSummaryMemory(llm=ChatOpenAI(), memory_key='message_log')


Chatbot is ready! Type 'exit' to end the conversation.


You:  I've recently moved to Australia. I've a new number and want to confirm it to receive SMS. How do I go about doing that? 


Taha's Bot: I'm sorry, but the SMS service is only available in the USA.


You:  None of the employees are able to view the schedules our manager created. Can you help?


...

Taha's Bot: What specific changes do you need to make to the shifts? It's important to understand your requirements thoroughly to provide the most accurate guidance.


You:  My employee took break using the break button but it does not deduct the break when I pull report.


Taha's Bot: The break time taken using the break button may not be deducted in the report because one of the managers may have disabled the break button from the settings.


You:  How do I sync Google calendar with Humanity?


Taha's Bot: To sync Google Calendar with Humanity, follow these steps:

1. In the Humanity app, go to the Shift Planning module.
2. Click on "Tools" in the top right corner.
3. Select "Schedule Sync" from the dropdown menu.
4. Copy the generated link or iCal sync URL.
5. Open your Google Calendar.
6. Click on the "+" button next to "Other Calendar" on the left-hand side.
7. Select "From URL" from the dropdown menu.
8. Paste the link or iCal sync URL from Humanity into the URL field.
9. Click on "Add calendar" to sync your Humanity schedule with Google Calendar.
